In [10]:
! pip install requests
! pip install pandas
! pip install numpy
! pip install datetime

DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.12/site-packages/acnportal-0.3.3-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.12/site-packages/acnportal-0.3.3-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.12/site-packages/acnportal-0.3.3-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.12/site-packages/acnportal-0.3.3-py3.12.egg is deprecated. pip 24.3 will

In [11]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime

# =============================================================================
# 1. LOAD THE DATA
# =============================================================================
df = pd.read_csv('/Users/huibvanderveen/Desktop/Pycharm Projects/ML_New/YEAR/CSV_2019/Combined_2019.csv')

df.columns


Index(['_id', 'clusterID', 'connectionTime', 'disconnectTime',
       'doneChargingTime', 'kWhDelivered', 'sessionID', 'siteID', 'spaceID',
       'stationID', 'timezone', 'userID', 'userInputs'],
      dtype='object')

In [9]:

# =============================================================================
# DATA CLEANING AND FILTERING
# =============================================================================
df.dropna(subset=['sessionID', 'connectionTime', 'disconnectTime', 'doneChargingTime', 'kWhDelivered'], inplace=True)

# Convert to datetime
for col in ['connectionTime', 'disconnectTime', 'doneChargingTime']:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Drop rows with failed datetime conversion
df.dropna(subset=['connectionTime', 'disconnectTime'], inplace=True)

# Calculate session duration in hours (using disconnectTime)
df['session_duration'] = (df['disconnectTime'] - df['connectionTime']).dt.total_seconds() / 3600.0

# Filter out non-positive and extreme durations
df = df[(df['session_duration'] > 0) & (df['session_duration'] < 12)]

# =============================================================================
# Normalize hour: shift such that 8:00 becomes 0 (the garage is in Los Angeles; times are in GMT)
# =============================================================================
df['hour'] = (df['connectionTime'].dt.hour - 8) % 24

# =============================================================================
# PLOTTING ARRIVAL RATE AND AVERAGE SESSION DURATION BY HOUR
# =============================================================================
# Compute arrival rate and average session duration
arrival_rate = df.groupby('hour').size()
avg_duration = df.groupby('hour')['session_duration'].mean()

# Convert to DataFrame for Plotly
metrics_df = pd.DataFrame({
    'hour': arrival_rate.index,
    'arrival_rate': arrival_rate.values,
    'avg_duration': avg_duration.values
})

# Plot arrival rate
fig1 = px.line(metrics_df, x='hour', y='arrival_rate', markers=True,
               labels={'hour': 'Hour of Day', 'arrival_rate': 'Number of Sessions'},
               title='Session Arrival Rate by Hour')
fig1.update_layout(xaxis=dict(tickmode='linear', tick0=0, dtick=1))

# Plot average session duration
fig2 = px.line(metrics_df, x='hour', y='avg_duration', markers=True,
               labels={'hour': 'Hour of Day', 'avg_duration': 'Average Duration (hours)'},
               title='Average Session Duration by Hour')
fig2.update_layout(xaxis=dict(tickmode='linear', tick0=0, dtick=1))

fig1.show()
fig2.show()

print("Arrival Rate by Hour:")
print(arrival_rate.sort_index())
print("\nAverage Session Duration by Hour (in hours):")
print(avg_duration.sort_index())

# =============================================================================
# COMPUTE CHARGING DURATION AND POWER OUTPUT
# =============================================================================
# Calculate charging duration in hours (time during which the car charged)
df['charging_duration'] = (df['doneChargingTime'] - df['connectionTime']).dt.total_seconds() / 3600.0

# Filter out non-positive charging durations
df = df[df['charging_duration'] > 0]

# Calculate average power output in kilowatts (kW)
df['power_output'] = df['kWhDelivered'] / df['charging_duration']

# =============================================================================
# COMPUTE SYSTEM DEMAND OVER TIME
# =============================================================================
# For each session, assume constant power output during charging.
# Create event records: at connectionTime, add the session's power_output; at doneChargingTime, subtract it.
df_start = df[['connectionTime', 'power_output']].copy()
df_start.rename(columns={'connectionTime': 'timestamp'}, inplace=True)

df_end = df[['doneChargingTime', 'power_output']].copy()
df_end.rename(columns={'doneChargingTime': 'timestamp'}, inplace=True)
df_end['power_output'] = -df_end['power_output']

# Combine start and end events, then sort by time
events_df = pd.concat([df_start, df_end]).sort_values('timestamp')

# Compute the cumulative sum to derive the instantaneous total power demand
events_df['total_power'] = events_df['power_output'].cumsum()

# Set the timestamp as index for resampling
events_df = events_df.set_index('timestamp')

# -----------------------------------------------------------------------------
# RESAMPLE DAILY, FILL MISSING DAYS WITH ZERO
# -----------------------------------------------------------------------------
# By default, .resample('D') creates a row for every day in the date range.
# We fill NaN with 0 to avoid gaps in the final line plot.
daily_avg = (events_df['total_power']
             .resample('D')
             .mean()
             .fillna(0)         # Fills days with no sessions as 0 kW
             .reset_index())

# =============================================================================
# PLOTTING AVERAGE SYSTEM DEMAND OVER THE YEAR
# =============================================================================
fig_demand = px.line(
    daily_avg,
    x='timestamp',
    y='total_power',
    labels={'timestamp': 'Date', 'total_power': 'Average System Demand (kW)'},
    title='Average kW Demand of the Garage Throughout the Year'
)
fig_demand.show()

print("Daily Average System Demand (kW):")
print(daily_avg)

Arrival Rate by Hour:
hour
0        4
1        3
2        1
3        2
4        9
5      141
6      223
7     1309
8     1879
9     1497
10     856
11     609
12     645
13     451
14     315
15     280
16     265
17     327
18     236
19     171
20     123
21      57
22      16
23      11
dtype: int64

Average Session Duration by Hour (in hours):
hour
0     7.591042
1     1.473704
2     8.681389
3     3.408750
4     2.761759
5     4.564145
6     6.868565
7     7.328102
8     7.097857
9     6.341437
10    5.476346
11    4.725435
12    3.940441
13    3.342833
14    2.429623
15    2.124362
16    2.048963
17    2.170816
18    2.512518
19    3.396353
20    3.899551
21    5.372130
22    5.385608
23    4.034091
Name: session_duration, dtype: float64


Daily Average System Demand (kW):
     timestamp  total_power
0   2019-01-01     5.668027
1   2019-01-02    42.164311
2   2019-01-03    25.537539
3   2019-01-04    28.997698
4   2019-01-05     7.179431
..         ...          ...
360 2019-12-27     7.486295
361 2019-12-28     3.138274
362 2019-12-29     3.718232
363 2019-12-30     3.614732
364 2019-12-31     2.428141

[365 rows x 2 columns]
